In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi
!pip install tqdm

In [4]:
# Load API Credentials
with open('/Users/dpatt/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [6]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [7]:
# set our API call parameters 
LOCATION = 'LA,CA'
TERM = 'Burritos'

In [8]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_CA_burritos.json"
JSON_FILE

'Data/results_in_progress_CA_burritos.json'

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_CA_burritos.json not found. Saving empty list to file.


In [10]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results

13400

In [13]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [14]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

670

In [15]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [16]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/670 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [17]:
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [18]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f) 

In [19]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_CA_burritos.json not found. Saving empty list to new file.
- 0 previous results found.


700

In [20]:

for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/700 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [21]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,1OXJaygVHAEGooC5gsH_Pg,la-esquina-los-angeles,La Esquina,https://s3-media1.fl.yelpcdn.com/bphoto/R8rSoL...,False,https://www.yelp.com/biz/la-esquina-los-angele...,417,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 34.0292220404164, 'longitude': -1...",[pickup],$$,"{'address1': '8905 Venice Blvd', 'address2': N...",+14242586396,(424) 258-6396,2224.279501
1,Lu2osODR3Cu9kTfaav3JiQ,brothers-cousins-tacos-los-angeles,Brothers Cousins Tacos,https://s3-media3.fl.yelpcdn.com/bphoto/vymeNo...,False,https://www.yelp.com/biz/brothers-cousins-taco...,355,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 34.02628342963417, 'longitude': -...",[],$,"{'address1': '11341 National Blvd', 'address2'...",,,2172.713987
2,9oQnnXSTo8i2TliFPktxLg,sonoritas-prime-tacos-los-angeles,Sonoritas Prime Tacos,https://s3-media2.fl.yelpcdn.com/bphoto/VH_w93...,False,https://www.yelp.com/biz/sonoritas-prime-tacos...,1805,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.04093, 'longitude': -118.44328}","[pickup, delivery]",$$,"{'address1': '2004 Sawtelle Blvd', 'address2':...",+13104449100,(310) 444-9100,3795.454943
3,mkhPLrwfYOJi_Y9ore3LqQ,el-barrio-neighborhood-tacos-los-angeles,El Barrio Neighborhood Tacos,https://s3-media1.fl.yelpcdn.com/bphoto/ylj-7u...,False,https://www.yelp.com/biz/el-barrio-neighborhoo...,311,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 34.0246774, 'longitude': -118.411...","[pickup, delivery]",$$,"{'address1': '3500 Overland Ave', 'address2': ...",+14246034073,(424) 603-4073,509.384088
4,VqxDHnbjfwwJXON7ERabOw,hermanito-los-angeles,Hermanito,https://s3-media1.fl.yelpcdn.com/bphoto/5vyBgJ...,False,https://www.yelp.com/biz/hermanito-los-angeles...,611,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 34.04048617, 'longitude': -118.44...","[pickup, delivery]",$$$,"{'address1': '2024 Sawtelle Blvd', 'address2':...",+14243882585,(424) 388-2585,3733.554572


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,5xFBQiB4Jcy9sI2fUHfcpg,mariscos-jalisco-los-angeles-3,Mariscos Jalisco,https://s3-media2.fl.yelpcdn.com/bphoto/I-bHSJ...,False,https://www.yelp.com/biz/mariscos-jalisco-los-...,1121,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 34.021051495081835, 'longitude': ...",[delivery],$,"{'address1': '3040 E Olympic Blvd', 'address2'...",+13235286701,(323) 528-6701,18233.615117
996,w7c062yyS5vPoLU2Nhkc4g,tacos-el-porvenir-los-angeles,Tacos El Porvenir,https://s3-media3.fl.yelpcdn.com/bphoto/sUsb35...,False,https://www.yelp.com/biz/tacos-el-porvenir-los...,5,"[{'alias': 'mexican', 'title': 'Mexican'}]",5.0,"{'latitude': 34.0200805664062, 'longitude': -1...",[delivery],NaN,"{'address1': '3000 Maple Ave', 'address2': '',...",+12137411260,(213) 741-1260,13205.747339
997,4ds8wN-SeL5Z1m9MABWhKQ,habanero-mexican-grill-san-gabriel,Habanero Mexican Grill,https://s3-media4.fl.yelpcdn.com/bphoto/VTHv97...,False,https://www.yelp.com/biz/habanero-mexican-gril...,321,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 34.1279029, 'longitude': -118.072...",[delivery],$,"{'address1': '9003 E Fairview Ave', 'address2'...",+16262861914,(626) 286-1914,33341.420804
998,u37xj9aNiFDpYdrYlJSh6Q,bubbakoos-burritos-lancaster-2,Bubbakoo's Burritos,https://s3-media2.fl.yelpcdn.com/bphoto/EKBIFV...,False,https://www.yelp.com/biz/bubbakoos-burritos-la...,133,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.0,"{'latitude': 34.6897114, 'longitude': -118.166...","[pickup, delivery]",$$,"{'address1': '2053 West Avenue J', 'address2':...",+16617231200,(661) 723-1200,77755.505549
999,7VKXmKrwl1eFiH2iwkoMzg,bros-tacos-mission-hills,Bros Tacos,https://s3-media3.fl.yelpcdn.com/bphoto/27mb8g...,False,https://www.yelp.com/biz/bros-tacos-mission-hi...,130,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",4.0,"{'latitude': 34.273967787503494, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '11220 Laurel Canyon Blvd', 'addr...",+18186390110,(818) 639-0110,28433.867830
